In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import albumentations as A

In [24]:
def add_noise_wrapper(color="black", amount=0.05):

    def add_noise(image, rows, cols):
        num_noise = np.ceil(amount * rows * cols)

        # random pixel
        coords = [
            np.random.randint(0, i - 1, int(num_noise)) for i in [rows, cols]
        ]

        if color == "black":
            image[coords[0], coords[1], :] = 0
        elif color == "white":
            image[coords[0], coords[1], :] = 255
        elif color == "blue":
            image[coords[0], coords[1], 0] = 125  # R
            image[coords[0], coords[1], 1] = 200  # G
            image[coords[0], coords[1], 2] = 230  # B

        return image

    return add_noise

In [ ]:
# -- 수정할 부분 --

img_dir = "/data/ephemeral/home/data/medical/img/train"  # train set 폴더 경로
save_dir = "/data/ephemeral/home/data/medical/aug"  # 저장할 폴더

img_idx = 0  # 증강 적용할 이미지 인덱스 (0 ~ 99)

color = "blue"  # 노이즈 색 (black / white / blue)
amount = 0.1  # 전체 픽셀에서 노이즈로 바꿀 비율 (0.0 ~ 1.0)
factor = -0.3  # 이미지 밝기 조절 (음수: 어둡게, 양수: 밝게)

# --------------

tf_noise = A.Lambda(image=add_noise_wrapper(color=color, amount=amount), p=1)
tf_bright = A.RandomBrightness(limit=(factor, factor), p=1)
tf_compose = A.Compose([tf_noise, tf_bright])

fnames = sorted(os.listdir(img_dir))
fname = fnames[img_idx]
img_path = os.path.join(img_dir, fname)

img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

fig = plt.figure(figsize=(14, 10))
ax1 = fig.add_subplot(1, 2, 1, xticks=[], yticks=[])
ax1.set_title("Original")
ax1.imshow(img)

aug_img = tf_noise(image=img)["image"]  # 노이즈만 추가
# aug_img = tf_bright(image=img)['image'] # 밝기만 조절
# aug_img = tf_compose(image=img)['image'] # 노이즈 추가 및 밝기 조절

ax2 = fig.add_subplot(1, 2, 2, xticks=[], yticks=[])
ax2.set_title("Transformed")
ax2.imshow(aug_img)

# plot보단 실제 저장된 이미지를 확인해보는 것이 더 정확함
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

cv2.imwrite(
    os.path.join(save_dir, fname), cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
)